In [1]:
import pytensor
import pytensor.tensor as pt
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from pymc.pytensorf import collect_default_updates
from pytensor.compile.mode import get_mode

In [2]:
az.style.use("arviz-darkgrid")

In [3]:
seed = sum(map(ord, "PyMC+AR2"))
rng = np.random.default_rng(seed)

In [4]:
lags = 2
trials = 100

coords = {
    "lags": range(-lags, 0),
    "steps": range(trials - lags),
    "trials": range(trials),
}
with pm.Model(coords=coords, check_bounds=False) as m:
    rho = pm.Normal("rho", 0, 0.2, dims=("lags",))
    sigma = pm.HalfNormal("sigma", .2)
    
    ar_init_obs = pm.MutableData("ar_init_obs", np.full((lags,), np.nan), dims=("lags",))
    ar_init = pm.Normal("ar_init", observed=ar_init_obs, dims=("lags",))

    def ar_step(*args):
        x_tm2, x_tm1, rho, sigma = args
        mu = x_tm1 * rho[0] + x_tm2 * rho[1]
        x = mu + pm.Normal.dist(sigma=sigma)
        return x, collect_default_updates(args, [x])
    
    ar_innov, ar_innov_updates = pytensor.scan(
        fn=ar_step,
        outputs_info=[{"initial": ar_init, "taps":range(-lags, 0)}],
        non_sequences=[rho, sigma],
        n_steps=trials-lags,
        strict=True,
    )

    ar_innov_obs = pm.MutableData("ar_innov_obs", np.full((trials-lags,), np.nan), dims=("steps",))
    ar_innov = m.register_rv(ar_innov, name="ar_innov", observed=ar_innov_obs, dims=("steps"))
    
    ar = pm.Deterministic("ar", pt.concatenate([ar_init, ar_innov], axis=-1), dims=("trials",))

NotImplementedError: MaskedArrays are not supported

In [5]:
lags = 2
trials = 100

coords = {
    "lags": range(-lags, 0),
    "steps": range(trials - lags),
    "trials": range(trials),
}
with pm.Model(coords=coords, check_bounds=False) as m:
    rho = pm.Normal("rho", 0, 0.2, dims=("lags",))
    
    mu = pm.Normal("mu")
    sigma = pm.HalfNormal("sigma", .2)
    
    ma_init_obs = pm.MutableData("ma_init_obs", np.full((lags,), np.nan), dims=("lags",))
    ma_init = pm.Normal("ma_init", observed=ma_init_obs, dims=("lags",))

    epsilon0 = ma_init[0] - mu
    epsilon1 = ma_init[1] - mu - rho[0] * epsilon0
    epsilon_init = pt.stack((epsilon0, epsilon1))
    
    def ma_step(*args):
        epsilon_tm2, epsilon_tm1, rho, mu, sigma = args
        mu = mu + epsilon_tm1 * rho[0] + epsilon_tm2 * rho[1]
        y = pm.Normal.dist(mu = mu, sigma=sigma)
        epsilon = y - mu
        return (epsilon, y), collect_default_updates(args, [y])
    
    [_, ma_innov], ma_innov_updates = pytensor.scan(
        fn=ma_step,
        outputs_info=[{"initial": epsilon_init, "taps":range(-lags, 0)}, None],
        non_sequences=[rho, mu, sigma],
        n_steps=trials-lags,
        strict=True,
    )

    ma_innov_obs = pm.MutableData("ma_innov_obs", np.full((trials-lags,), np.nan), dims=("steps",))
    ma_innov = m.register_rv(ma_innov, name="ma_innov", observed=ma_innov_obs, dims=("steps"))
    
    ma = pm.Deterministic("ma", pt.concatenate([ma_init, ma_innov], axis=-1), dims=("trials",))

NotImplementedError: MaskedArrays are not supported